In [ ]:
from rich import print as rprint

### Basic LLM call


#### Sync


In [ ]:
from tinyloop.inference.litellm import LLM

llm = LLM(model="openai/gpt-3.5-turbo", temperature=0.1)

response = llm(prompt="Hello, how are you?")
rprint(response)

In [ ]:
llm.get_history()

#### Async


In [ ]:
from tinyloop.inference.litellm import LLM

llm = LLM(model="openai/gpt-3.5-turbo", temperature=0.1)

response = await llm.acall(prompt="Hello, how are you?")
rprint(response)

In [ ]:
llm.get_history()

### Structured outputs


#### Sync


In [ ]:
from tinyloop.inference.litellm import LLM
from pydantic import BaseModel

llm = LLM(
    model="openai/gpt-4.1-nano",
    temperature=0.1,
)


class CalendarEvent(BaseModel):
    name: str
    date: str
    participants: list[str]


class EventsList(BaseModel):
    events: list[CalendarEvent]


response = llm(
    prompt="List 5 important events in the XIX century", response_format=EventsList
)
rprint(response)

#### Async


In [ ]:
from tinyloop.inference.litellm import LLM

llm = LLM(model="openai/gpt-3.5-turbo", temperature=0.1)

response = await llm.acall(prompt="Hello, how are you?")
rprint(response)

In [ ]:
from tinyloop.inference.litellm import LLM
from pydantic import BaseModel

llm = LLM(
    model="anthropic/claude-3-5-haiku-20241022",
    temperature=0.1,
)


class CalendarEvent(BaseModel):
    name: str
    date: str
    participants: list[str]


class EventsList(BaseModel):
    events: list[CalendarEvent]


response = await llm.acall(
    prompt="List 5 important events in the XIX century", response_format=EventsList
)
rprint(response)

In [ ]:
from tinyloop.inference.litellm import LLM
from pydantic import BaseModel

llm = LLM(
    model="openrouter/google/gemini-2.5-flash",
    temperature=0.1,
)


class CalendarEvent(BaseModel):
    name: str
    date: str
    participants: list[str]


class EventsList(BaseModel):
    events: list[CalendarEvent]


response = await llm.acall(
    prompt="List 5 important events in the XIX century", response_format=EventsList
)
rprint(response)

### Vision


#### Sync


##### From PIL


In [ ]:
from tinyloop.inference.litellm import LLM
from PIL import Image as PILImage
from tinyloop.features.vision import Image

llm = LLM(
    model="openai/gpt-4.1-nano",
    temperature=0.1,
)


# Read a local JPG file as PIL image
pil_image = PILImage.open("sutton.jpg")

# Create tinyloop Image from PIL image
image = Image.from_PIL(pil_image)


response = llm(prompt="Describe the image", images=[image])

rprint(response)

##### From File


In [ ]:
from tinyloop.inference.litellm import LLM
from PIL import Image as PILImage
from tinyloop.features.vision import Image

llm = LLM(
    model="anthropic/claude-3-7-sonnet-20250219",
    temperature=0.1,
)

# Create tinyloop Image from PIL image
image = Image.from_file("sutton.jpg")


response = llm(prompt="Describe the image", images=[image])

rprint(response)

##### From URL


In [ ]:
from tinyloop.inference.litellm import LLM
from PIL import Image as PILImage
from tinyloop.features.vision import Image
import litellm

llm = LLM(
    model="openrouter/google/gemini-2.5-pro",
    temperature=0.1,
)
litellm._turn_on_debug()

# Create tinyloop Image from PIL image
url = "https://images.ctfassets.net/cnu0m8re1exe/2xdqQSvfebktASbHvILYH5/fcc91130ad1ff329765595b669549d8d/Meet-Jumping-Spider-Adorable-Arachnid.jpg?fm=jpg&fl=progressive&w=660&h=433&fit=fill"
image = Image.from_url(url)


response = llm(prompt="Describe the image", images=[image])

rprint(response)

#### Async


##### From PIL


In [ ]:
from tinyloop.inference.litellm import LLM
from PIL import Image as PILImage
from tinyloop.features.vision import Image
from rich import print as rprint

llm = LLM(
    model="openrouter/google/gemini-2.5-flash",
    temperature=0.1,
)


# Read a local JPG file as PIL image
pil_image = PILImage.open("sutton.jpg")

# Create tinyloop Image from PIL image
image = Image.from_PIL(pil_image)


response = await llm.acall(prompt="Describe the image", images=[image])

rprint(response)

##### From File


In [ ]:
from tinyloop.inference.litellm import LLM
from PIL import Image as PILImage
from tinyloop.features.vision import Image

llm = LLM(
    model="openai/gpt-4.1-nano",
    temperature=0.1,
)

# Create tinyloop Image from PIL image
image = Image.from_file("sutton.jpg")


response = await llm.acall(prompt="Describe the image", images=[image])

rprint(response)

##### From URL


In [ ]:
from tinyloop.inference.litellm import LLM
from PIL import Image as PILImage
from tinyloop.features.vision import Image

llm = LLM(
    model="anthropic/claude-3-7-sonnet-20250219",
    temperature=0.1,
)

# Create tinyloop Image from PIL image
url = "https://images.ctfassets.net/cnu0m8re1exe/2xdqQSvfebktASbHvILYH5/fcc91130ad1ff329765595b669549d8d/Meet-Jumping-Spider-Adorable-Arachnid.jpg?fm=jpg&fl=progressive&w=660&h=433&fit=fill"
image = Image.from_url(url)


response = await llm.acall(prompt="Describe the image", images=[image])

rprint(response)

### Function calling


In [ ]:
from tinyloop.inference.litellm import LLM
import json
from tinyloop.features.function_calling import Tool


def get_current_weather(location: str, unit: str):
    """Get the current weather in a given location

    Parameters
    ----------
    location : str
        The city and state, e.g. San Francisco, CA
    unit : str {'celsius', 'fahrenheit'}
        Temperature unit

    Returns
    -------
    str
        a sentence indicating the weather
    """
    if location == "Boston, MA":
        return "The weather is 12F"
    return f"Weather in {location} is sunny"


llm = LLM(
    model="openai/gpt-4.1-nano",
    temperature=0.1,
)

weather_tool = Tool(get_current_weather)

inference = llm(
    prompt="What is the weather in Boston, MA?",
    tools=[weather_tool],
)

for tool_call in inference.raw_response.choices[0].message.tool_calls:
    tool_name = tool_call.function.name
    tool_args = json.loads(tool_call.function.arguments)
    print(f"Tool: {tool_name}")
    print(f"Args: {tool_args}")
    print(weather_tool(**tool_args))

In [ ]:
from tinyloop.inference.litellm import LLM
import json
from tinyloop.features.function_calling import Tool


def get_current_weather(location: str, unit: str):
    """Get the current weather in a given location

    Parameters
    ----------
    location : str
        The city and state, e.g. San Francisco, CA
    unit : str {'celsius', 'fahrenheit'}
        Temperature unit

    Returns
    -------
    str
        a sentence indicating the weather
    """
    if location == "Boston, MA":
        return "The weather is 12F"
    return f"Weather in {location} is sunny"


llm = LLM(
    model="openrouter/google/gemini-2.5-flash",
    temperature=0.1,
)

weather_tool = Tool(get_current_weather)

inference = llm(
    prompt="What is the weather in Boston, MA (in fahrenheit)?",
    tools=[weather_tool],
)

for tool_call in inference.raw_response.choices[0].message.tool_calls:
    tool_name = tool_call.function.name
    tool_args = json.loads(tool_call.function.arguments)
    print(f"Tool: {tool_name}")
    print(f"Args: {tool_args}")
    print(weather_tool(**tool_args))